# Recording

> Storing data

In [ ]:
#|default_exp recording

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from AIsaac.utils import *
from AIsaac.dataloaders import *
from AIsaac.models import *
from AIsaac.initialization import *
from AIsaac.trainer import *
from AIsaac.training import *

from datetime import datetime, timedelta
import torchvision.transforms.functional as TF,torch.nn.functional as F
import math, time

import matplotlib.pyplot as plt
import matplotlib as mpl
import fastcore.all as fc
import torch
from torch import nn, Tensor
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import pandas as pd , numpy as np
from torcheval.metrics import MulticlassAccuracy,Mean
from torch.optim.lr_scheduler import ExponentialLR

import dill as pickle
from fastprogress.fastprogress import master_bar, progress_bar
import inspect
import torchinfo

/home/python3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#|hide
import logging

In [ ]:
#|hide
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
# mpl.rcParams['image.cmap'] = 'gray'

logging.disable(logging.WARNING)

set_seed(42)


In [ ]:
xmean,xstd = 0.28, 0.35
@inplace
def transformi(b): b['image'] = [(TF.to_tensor(o)-xmean)/xstd for o in b['image']]

_dataset = sample_dataset_dict(load_dataset('fashion_mnist').with_transform(transformi))

100%|██████████| 2/2 [00:00<00:00, 354.55it/s]


In [ ]:
dls = DataLoaders.from_dataset_dict(_dataset, 256, num_workers=4)

## Core

In [ ]:
#| export
class MetricsCB(Callback):
    '''Callback to track train/valid loss + metrics'''
    def __init__(self, **metrics):
        self.metrics = metrics
        self.losses = {'train':Mean(),'valid':Mean()}
        self.metrics_epoch,self.losses_epoch,self.losses_batch = [],[],[]
            
    def after_batch(self,trainer):
        '''stores losses and metrics for batch'''        
        self.losses[f"{'train' if trainer.training else 'valid'}"].update(to_cpu(trainer.loss),weight=len(trainer.batch[1]))
        if not trainer.training:
            preds,batch = map(to_cpu,[trainer.preds,trainer.batch[1]])    
            for k in self.metrics: self.metrics[k].update(preds,batch)
        self.losses_batch.append({'training':trainer.training,'loss':to_cpu(trainer.loss)})
            
    def cleanup_epoch(self,trainer):
        '''compute metrics and append to epoch stats and display'''
        if not trainer.training:
            self.metrics_epoch.append({name:float(metric.compute()) for name, metric in self.metrics.items()})
            self.losses_epoch.append({name:float(metric.compute()) for name, metric in self.losses.items()})

            for metric in self.metrics.values(): metric.reset()
            for metric in self.losses.values(): metric.reset()

In [ ]:
trainer = Trainer(dls,
                  nn.CrossEntropyLoss(), 
                  torch.optim.Adam, 
                  get_model_conv(), 
                  callbacks=[BasicTrainCB(),MetricsCB(accuracy=MulticlassAccuracy()), DeviceCB()])

In [ ]:
trainer.fit()

In [ ]:
fc.test_eq(pd.DataFrame(trainer.MetricsCB.losses_epoch).shape,(3,2))
fc.test_eq(pd.DataFrame(trainer.MetricsCB.metrics_epoch).shape,(3,1))
fc.test_eq(pd.DataFrame(trainer.MetricsCB.losses_batch).shape,(9,2))

In [ ]:
#| export
class ProgressCB(Callback):
    '''Callback to display progress while training'''
        
    def before_fit(self,trainer):
        '''Initialize Fit Progress Bar'''
        trainer.epochs = master_bar(trainer.epochs)
        
    def before_epoch(self,trainer):
        '''Initialize Epoch Progress Bar'''
        trainer.batches = progress_bar(trainer.batches,parent=trainer.epochs)
        trainer.epochs.child.comment = "Training" if trainer.training else "Validation"
        
    def cleanup_epoch(self,trainer):
        '''Display Loss and Metric'''
        if not trainer.training: 
            df = pd.concat(map(pd.DataFrame,[trainer.MetricsCB.losses_epoch,trainer.MetricsCB.metrics_epoch]),axis=1).tail(1)
            if trainer.epoch != 0: df = df.style.set_table_styles([{'selector': 'thead', 'props': [('display', 'none')]}])
            display(df)

In [ ]:
trainer = Trainer(dls,
                  nn.CrossEntropyLoss(), 
                  torch.optim.Adam, 
                  get_model_conv(), 
                  callbacks=[BasicTrainCB(),MetricsCB(accuracy=MulticlassAccuracy()), DeviceCB(),OneBatchCB(),ProgressCB()])

In [ ]:
trainer.fit()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()